In [0]:
pip install pyspark

Python interpreter will be restarted.
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425366 sha256=d396f614033a71cbf81f21febb4c12765c19a947da10909ff0130c59cdcfae1d
  Stored in directory: /root/.cache/pip/wheels/57/bd/14/ce9e21f2649298678d011fb8f71ed38ee70b42b94fef0be142
Successfully built pyspark
Python interpreter will be restarted.


In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql.functions import desc

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
# Load the Spotify dataset into a DataFrame
spotify_data = spark.read.csv("/FileStore/tables/train.csv", header=True, inferSchema=True)

In [0]:
spotify_data.show(5)

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   False|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|

Query 1: Identifying high Potential Genres for Marketing and promotion

In [0]:
# Cache the DataFrame for better performance
spotify_data.cache()
# Find the top genres based on the average popularity of tracks within each genre
high_potential_genres_query = (
 spotify_data
 .groupBy("track_genre")
 .agg({"popularity": "avg"})
 .orderBy(desc("avg(popularity)"))
 .limit(10)
)
high_potential_genres_query.show() 


+-----------+------------------+
|track_genre|   avg(popularity)|
+-----------+------------------+
|   pop-film|59.287575150300604|
|      k-pop|            56.896|
|      chill|            53.651|
|        sad|            52.379|
|     grunge|            49.594|
|     indian|            49.539|
|      anime|            48.772|
|        emo|            48.128|
|  sertanejo|            47.866|
|        pop|            47.576|
+-----------+------------------+



Query2: Identify Top Artists and Genres by Average Popularity

In [0]:
top_artists_genres_query = (
 spotify_data
 .groupBy("artists", "track_genre")
 .agg({"popularity": "avg"})
 .orderBy(desc("avg(popularity)"))
 .limit(10)
)
top_artists_genres_query.show() 

+--------------------+-----------+---------------+
|             artists|track_genre|avg(popularity)|
+--------------------+-----------+---------------+
|Sam Smith;Kim Petras|      dance|          100.0|
|Sam Smith;Kim Petras|        pop|          100.0|
|    Bizarrap;Quevedo|    hip-hop|           99.0|
|       Manuel Turizo|      latin|           98.0|
|       Manuel Turizo|     latino|           98.0|
|       Manuel Turizo|     reggae|           98.0|
|       Manuel Turizo|  reggaeton|           98.0|
|Bad Bunny;Chencho...|      latin|           97.0|
|Bad Bunny;Chencho...|     reggae|           97.0|
|Bad Bunny;Chencho...|     latino|           97.0|
+--------------------+-----------+---------------+



Query3: Identify the Most Common Keys and Modes

Knowing the most common keys and modes in your dataset can help in creating
playlists. 

In [0]:
common_keys_modes_query = (
 spotify_data
 .groupBy("key", "mode")
 .count()
 .orderBy(desc("count"))
)
common_keys_modes_query.show(10) 


+---+----+-----+
|key|mode|count|
+---+----+-----+
|  0|   1|10179|
|  7|   1|10130|
|  2|   1| 9052|
|  1|   1| 7164|
|  9|   1| 6857|
|  8|   1| 5436|
|  5|   1| 5336|
| 11|   0| 5145|
|  4|   0| 4724|
|  9|   0| 4453|
+---+----+-----+
only showing top 10 rows

